In [1]:
import cobra
import numpy as np

In [2]:
modelo=cobra.io.read_sbml_model("./iSB619.xml")

In [ ]:
modelo

In [3]:
n=len(modelo.reactions)
m=len(modelo.metabolites)

In [4]:
modelo.compartments

{'c': 'cytosol', 'e': 'extracellular space'}

In [5]:
S=cobra.util.array.create_stoichiometric_matrix(modelo, 'dense');S

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
len(S)*len(S[0])

In [ ]:
cont=0
for m in modelo.metabolites:
    if m.compartment=='c':
        print(m.id)
        cont+=1
print("Hay",cont)

In [ ]:
cont=0
for m in modelo.metabolites:
    if m.compartment=='e':
        print(m.id)
        cont+=1
print("Hay",cont)

In [ ]:
cont=0
for r in modelo.reactions:
    if "EX_" in r.id:
        print(r.id,r)
        cont+=1
print("Hay",cont)

In [ ]:
reversibles=[]
irreversibles=[]
for i in range(len(modelo.reactions)):
    r=modelo.reactions[i]
    if r.lower_bound*r.upper_bound<0:
        reversibles.append(i)
    else:
        irreversibles.append(i)
print("Hay",len(reversibles),"reversibles y",len(irreversibles),"irreversibles")

In [ ]:
internas=[]
for i in range(len(S)):
    fila=S[i]
    pos=False
    neg=False
    for j in range(len(fila)):
        r=modelo.reactions[j]
        if not fila[j]==0 :
            if j in reversibles:
                pos=True
                neg=True
            elif  r.lower_bound>=0:
                if fila[j]>0:
                    pos=True
                else:
                    neg=True
            else:
                if fila[j]>0:
                    neg=True
                else:
                    pos=True
        if pos and neg:
            internas.append(i)
            break
externas=set(range(len(modelo.metabolites))).difference(set(internas))
externas=list(externas)
externas.sort()
print(len(internas),len(externas))
for m in externas:
    print(m,modelo.metabolites[m].id)

In [ ]:
solucion=modelo.optimize()

In [ ]:
soporte=[]
for j in range(n):
    r=modelo.reactions[j].id
    if abs(solucion.fluxes[r])>10**-12:
        soporte.append(j)
noSoporte=set(range(n)).difference(set(soporte))
noSoporte=list(noSoporte)
noSoporte.sort()
print(soporte)

In [ ]:
print("hay",len(modelo.metabolites),"metabolitos, por tanto hay",len(modelo.constraints),"restricciones")

In [ ]:
for j in range(n):
    r=modelo.reactions[j]
    if r.lower_bound*r.upper_bound>0:
        print(j,r.id,r.lower_bound,r.upper_bound)

In [ ]:
bloqueadas=set()
with modelo:
    coefficients = dict()
    r=modelo.reactions.get_by_id("EX_glc__D_e")
    coefficients[r.forward_variable] = 1
    constraint = modelo.problem.Constraint(0, lb=1, ub=1)
    modelo.add_cons_vars(constraint)
    modelo.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    for i in range(len(modelo.reactions)):
        modelo.objective=modelo.reactions[i]
        sol1=modelo.optimize(objective_sense="minimize")
        sol2=modelo.optimize(objective_sense="maximize")
        print(i,modelo.reactions[i].id,modelo.reactions[i].reversibility,sol1.objective_value,sol2.objective_value)
        if sol1.objective_value==0 and sol2.objective_value==0:
            bloqueadas.add(i)

In [ ]:
print("Hay",len(bloqueadas),"reacciones bloqueadas")
val={}
for j in bloqueadas:
    r=modelo.reactions[j].id
    val[r]=1

In [ ]:
val={}
direcciones={}
with modelo:
    coefficients = dict()
    r=modelo.reactions.get_by_id("EX_glc__D_e")
    coefficients[r.forward_variable] = 1
    constraint = modelo.problem.Constraint(0, lb=1, ub=1)
    modelo.add_cons_vars(constraint)
    modelo.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    for i in range(n):
        r=modelo.reactions[i]
        modelo.objective=modelo.reactions[i]
        sol1=modelo.optimize(objective_sense="minimize")
        sol2=modelo.optimize(objective_sense="maximize")
        if r.reversibility:
            if sol1.objective_value<0 and sol2.objective_value<=0:
                val[r.id]=-1
                direcciones[i]=-1
            elif sol1.objective_value>=0 and sol2.objective_value>0:
                val[r.id]=1
                direcciones[i]=1
            else:
                val[r.id]=0
                direcciones[i]=0
        else:
            if sol1.objective_value<0 and sol2.objective_value<=0:
                direcciones[i]=-1
            elif sol1.objective_value>=0 and sol2.objective_value>0:
                direcciones[i]=1
            else:
                direcciones[i]=0

In [ ]:
implicaciones={}
for i in range(n):  
    if not i in bloqueadas:
        mias=set()
        for j in range(n):
            with modelo as modelo2:
                r=modelo2.reactions[i]
                if not i==j:
                    if direcciones[i]==1:
                        r.lower_bound=0.1
                        r2=modelo2.reactions[j]
                        r2.lower_bound=0
                        r2.upper_bound=0
                        sol=modelo2.optimize(objective_sense="minimize")
                        if sol.status=="infeasible":
                            mias.add(j)
                    if direcciones[i]==-1:
                        r.upper_bound=-0.1
                        r2=modelo2.reactions[j]
                        r2.lower_bound=0
                        r2.upper_bound=0
                        sol=modelo2.optimize(objective_sense="minimize")
                        if sol.status=="infeasible":
                            mias.add(j)
                    if direcciones[i]==0:
                        a=r.lower_bound
                        r.lower_bound=0.1
                        r2=modelo2.reactions[j]
                        r2.lower_bound=0
                        r2.upper_bound=0
                        sol1=modelo2.optimize()
                        r.upper_bound=-0.1
                        r.lower_bound=a
                        r2=modelo2.reactions[j]
                        r2.lower_bound=0
                        r2.upper_bound=0
                        sol2=modelo2.optimize()
                        if sol1.status=="infeasible" and sol2.status=="infeasible":
                            mias.add(j)
        implicaciones[i]=mias

        print(implicaciones)

In [ ]:
equivalencias=set()
for i in range(n):
    if not i in bloqueadas:
        for j in range(n):
            if not j in bloqueadas:
                if j in implicaciones[i] and i in implicaciones[j]:
                    equivalencias.add(frozenset([i,j]))
print(equivalencias)

In [6]:
def check(flujos,S):
    modo=[flujos[r] for r in flujos.keys()]
    e=10**-6
    sop=[j for j in range(len(modo)) if abs(modo[j])>10**-8]
    noSop=[j for j in range(len(modo)) if abs(modo[j])<=10**-8]
    val=np.dot(S,modo)
    if min(val)<-e or max(val)>e:
        print("Sv no vale 0")
        return False
    for j in range(len(modo)):
        r=modelo.reactions[j]
        if modo[j]<r.lower_bound or modo[j]>r.upper_bound:
            print("v no cumple v_i entre su maximo y minimo")
            print("Reaccion",j,r.id, "con valor",modo[j],"no entre",r.lower_bound,r.upper_bound)
            return False
    S2=np.delete(S,noSop,1)
    if not (len(sop)==np.linalg.matrix_rank(S2)+1):
        print("rango",np.linalg.matrix_rank(S2),len(sop))
        return False
    return True

In [7]:
solucion=modelo.optimize(objective_sense="minimize")
flujos=solucion.fluxes

In [8]:
check(flujos,S)

ValueError: zero-size array to reduction operation maximum which has no identity

In [10]:
soportes=set()
cont=0
contFallos=0


for i in range(n):
    cont+=1
    r=modelo.reactions[i]
    modelo.objective=r
    solucion=modelo.optimize(objective_sense="minimize")
    flujos=solucion.fluxes
    if not True:
        contFallos+=1
    else:
        modo=[flujos[r] for r in flujos.keys()]
        sop=[j for j in range(len(modo)) if abs(modo[j])>10**-6]
        if len(sop)>0 and not frozenset(sop) in soportes:
            soportes.add(frozenset(sop))

In [11]:
print(cont,len(soportes),contFallos)

743 131 0


In [12]:
import pickle

with open("./soportes.txt", 'wb') as filehandle:
    pickle.dump(soportes, filehandle)